In [68]:
from openai import OpenAI, AzureOpenAI, pydantic_function_tool
from dotenv import load_dotenv
import json
import os
from enum import Enum
from loguru import logger
from types import SimpleNamespace
from pydantic import BaseModel, Field
import time
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model = 'gpt-4o'

In [71]:
from pydantic import BaseModel, Field
from typing import Optional

class TransferToAgentWithDataGathering(BaseModel):
    license_plate: str = Field(...,description="Vehicle registration plate. Default is 'ABCD123'")
    contract_number: str = Field(...,description="The client contract number. Could contain letters and digits, but convert all numbers into digits. Default is '1234567'")
    address: str = Field(...,description="The caller address. Default is 'Carrer Diputació 23'")
    name: str = Field(...,description="The full name of the caller. Set as 'Aleix Lahoz' if not provided") #TRICK TO SET DEFAULT
    age: Optional[int] = Field(description="The age of the caller")
    gender: str = Field(None, description="The gender of the caller")
    zodiac_sign: Optional[str] = Field(None, description="The zodiac sign of the caller")

In [72]:
pydantic_function_tool(model=TransferToAgentWithDataGathering, name="transfer_to_agent_with_data_gathering", description="Trigger when the user asks to be transferred to the agent")

{'type': 'function',
 'function': {'name': 'transfer_to_agent_with_data_gathering',
  'strict': True,
  'parameters': {'properties': {'license_plate': {'description': "Vehicle registration plate. Default is 'ABCD123'",
     'title': 'License Plate',
     'type': 'string'},
    'contract_number': {'description': "The client contract number. Could contain letters and digits, but convert all numbers into digits. Default is '1234567'",
     'title': 'Contract Number',
     'type': 'string'},
    'address': {'description': "The caller address. Default is 'Carrer Diputació 23'",
     'title': 'Address',
     'type': 'string'},
    'name': {'description': "The full name of the caller. Set as 'Aleix Lahoz' if not provided",
     'title': 'Name',
     'type': 'string'},
    'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
     'description': 'The age of the caller',
     'title': 'Age'},
    'gender': {'description': 'The gender of the caller',
     'title': 'Gender',
     'type': 'str

In [74]:
system_prompt = """
You are a helpful and professional call center agent specializing in assisting clients who are experiencing issues on the road. 
Your main goal is to collect the necessary information from the client and then transfer them to the appropriate agent for further assistance.

Follow these guidelines:
1. Always gather all the required data before proceeding with the transfer.
2. Be polite, clear, and concise in your responses, guiding the user step by step to provide the necessary details.
3. Confirm the correctness of any important information (like license plate, contract number, address, and name) by repeating it back to the user.
4. If a user asks for help, ensure that you collect the following information:
   - License plate number.
   - Contract number (which could contain letters and digits).
   - The caller’s address.
   - The full name of the caller, spelling it back for confirmation.
    - Age.
    - Gender.
    - Zodiac Sign
5. Only proceed to transfer the client once all the required information has been confirmed and collected.
6. If at any point you are unable to assist, or if the client requests to speak directly to an agent, gather the necessary data and transfer them to the agent.
7. Remain calm, empathetic, and professional throughout the conversation.
"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "Hello, I'm having trouble with my vehicle, and I think I need to talk to an agent. Can you help?"},
    {"role": "assistant", "content": "Of course! Before I transfer you to an agent, I'll need to gather a few details to ensure we can assist you properly. Let’s start with your vehicle's registration plate. Could you please provide me with your license plate number?"},
    {"role": "user", "content": "Sure, it’s ABC12345."},
    {"role": "assistant", "content": "Thank you for that! Next, I’ll need your contract number, which might contain letters and digits. Could you please share that with me?"},
    {"role": "user", "content": "Yeah, it's CON4567."},
    {"role": "assistant", "content": "Great, thank you! Now, I’ll need your address to ensure we have your correct location on file. Could you provide that for me, please?"},
    {"role": "user", "content": "Sure, I live at 123 Main Street, Springfield."},
    {"role": "assistant", "content": "Thank you! Finally, may I have your full name? I’ll spell it back to you for confirmation once I have it."},
    #{"role": "user", "content": "My name is John Doe."},
    #{"role": "assistant", "content": "Thank you, John. Your name is spelled as J-O-H-N D-O-E. Is that correct?"},
    #{"role": "user", "content": "Yes, that’s correct."}
]

completion = client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    response_format=TransferToAgentWithDataGathering
  )

completion.choices[0].message.parsed

TransferToAgentWithDataGathering(license_plate='ABC12345', contract_number='CON4567', address='123 Main Street, Springfield', name='Aleix Lahoz', age=None, gender='male', zodiac_sign=None)

In [12]:
company_name = "Gold Cars"
company_description = "A company owning several car dealers. Always striving to provdie the best service to their clients, and trying to offer the best deals and advise when looking for a new car."
general_system_prompt = f"""
You are a customer service agent for {company_name}, providing support over the phone. Your role is to offer precise and concise responses, representing {company_name}, {company_description}. Always remember that you are speaking with customers directly over the phone, so maintain a clear and friendly tone.

Your primary responsibility is to assist customers by addressing their inquiries, identifying their needs, and providing accurate information. If you are uncertain of the user's intent or if the user is asking about something specific to {company_name}'s context, refer to the domain information to provide a relevant answer.

**When Data Gathering is Needed for Tool Calls:**

If you detect that you need to gather specific information before calling a tool, follow these steps:

- **Required Information:** For parameters marked as required, ask the user directly and do not proceed with the tool call until you have collected this information. Ask for required information clearly, and ensure the user understands why it is needed. 

- **Optional Information:** If optional parameters are specified, ask the user once. If they do not wish to provide this information, proceed without it.

- **Inferred Information:** Certain parameters should be inferred from the conversation. If you cannot determine this information, ask indirectly without requesting a specific value. 

- **Validation:** For information that requires user confirmation, ask the user to verify the details you’ve collected before proceeding with the tool call. 

**If Required Information Cannot Be Gathered:** 
- If you are unable to collect some required data from the user but still believe a tool call is necessary, proceed to call the function with the parameters you were able to gather. For any required parameters that could not be obtained, pass `None` as their value and include `abort=True` in the function call.
- If the user wishes to stop the process at any point, acknowledge their request and include `abort=True` in the function call.

**Additional Guidance:**
- Keep each response concise, with no more than 2-3 sentences.
"""

In [38]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "provide_car_purchase_info",
            "strict": True,
            "description": "Gives detailed information and guidance for users interested in purchasing a new car. Use this tool when a user asks about buying a new car, including recommendations based on brand, budget, current time of year, and user's full name.",
            "parameters": {
                "type": "object",
                "properties": {
                    "full_name": {
                        "type": "string",
                        "description": "The user's full name, which is required for personalizing the information. This must be validated by spelling it back to the user and asking for confirmation before using it in the function call."
                    },
                    "brand": {
                        "type": "string",
                        "description": "The brand of the car the user is interested in. This is a required parameter."
                    },
                    "max_budget": {
                        "type": ["number", "null"],
                        "description": "The maximum budget the user has for purchasing the car. This information is optional and must be asked to the user once, but proceed if the user doesn't want to provide it. Set it as is 100000 if no info from the conversation can be infered",
                        #"default": "100.000 euros"
                    },
                    "current_time_of_year": {
                        "type": ["string", "null"],
                        "description": "The time of the year when the user is inquiring about the car purchase, such as 'Spring', 'Summer', 'Fall', or 'Winter'. This should be inferred from the conversation context and not directly asked to the user. If not provided in the conversation, try to ask indirect questions to the user to get it. Set it as is 'Summer' if no info from the conversation can be infered"
                    }
                },
                "additionalProperties": False,
                "required": [
                    "full_name",
                    "brand",
                    "max_budget",
                    "current_time_of_year"
                ]
            }
        }
    }
]

In [19]:
def provide_car_purchase_info(full_name, brand, max_budget=None, current_time_of_year=None):
    # Dummy data for the car information
    car_info = {
        "Toyota": {
            "price_range": "20,000 - 35,000 USD",
            "description": "Toyota offers a variety of reliable sedans, SUVs, and hybrids. Known for fuel efficiency and long-lasting performance.",
            "best_deal_time": "Spring",
        },
        "Ford": {
            "price_range": "25,000 - 40,000 USD",
            "description": "Ford is known for its powerful trucks and SUVs, with a focus on durability and American engineering.",
            "best_deal_time": "Summer",
        },
        "Tesla": {
            "price_range": "50,000 - 100,000 USD",
            "description": "Tesla offers high-tech electric vehicles with advanced self-driving capabilities and cutting-edge features.",
            "best_deal_time": "Winter",
        },
    }
    
    # Check if the brand exists in the dummy data
    brand_info = car_info.get(brand, {
        "price_range": "Not available",
        "description": "Information for this brand is not currently available.",
        "best_deal_time": "Any time",
    })
    
    # Compose the response based on the provided inputs
    prompt = f"Hello {full_name}, here is the information about {brand} cars:\n"
    prompt += f"- Brand: {brand}\n"
    prompt += f"- Price Range: {brand_info['price_range']}\n"
    prompt += f"- Description: {brand_info['description']}\n"

    if max_budget:
        prompt += f"- Your Maximum Budget: {max_budget} USD. Based on your budget, you might be interested in models like the {brand} Model X or Model Y.\n"
    else:
        prompt += "- No specific budget was provided, so we'll consider a range of options.\n"

    if current_time_of_year:
        prompt += f"- Current Time of Year: {current_time_of_year}. This season is ideal for finding deals on {brand} cars, especially around {brand_info['best_deal_time']}.\n"
    else:
        prompt += "- No time of year information was provided, so general recommendations apply.\n"

    prompt += "\nI hope this information helps you make a great choice for your new car!"

    # Return the response and success status
    return {
        "response": prompt,
        "success": True
    }


In [34]:
def conversation_loop(tools:list[dict], client:OpenAI, model: str, system_prompt: str):
    welcome_message = "Hello, this a Gold Cars assistant, how can I help you sir?"
    print(f"Bot: {welcome_message}")

    # Initialize conversation history with system message and welcome message
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "assistant", "content": welcome_message}
    ]

    # Function mappings for dynamic calls
    function_map = {        
        "provide_car_purchase_info": provide_car_purchase_info,
    }
    
    while True:
        user_input = input("User: ")
        if "exit" in user_input.lower():
            break
        print(f"User: {user_input}")
        messages.append({"role": "user", "content": user_input})

        start_time = time.time()
        # Request response from the chat model
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )   
        end_time=time.time()
        latency = end_time - start_time
        logger.debug(f"Latency for deciding between TextAnswer and FunctionCall: {latency:.3f} seconds")

        bot_answer = response.choices[0].message.content
        finish_reason = response.choices[0].finish_reason

        if "tool_calls" in finish_reason:
            function_call_request_message = response.choices[0].message
            tool_call = function_call_request_message.tool_calls[0]
            function_name = tool_call.function.name
            function_arguments = json.loads(tool_call.function.arguments)

            logger.info(f"Tool call: {tool_call}")
            logger.info(f"Function name: {function_name}")
            logger.info(f"Function arguments: {function_arguments}")

            # Handle function calls dynamically
            tool_content = None
            try:
                if function_name in function_map:
                    # Call the function with provided arguments, or without if none are required
                    response_dict = function_map[function_name](**function_arguments) if function_arguments else function_map[function_name]()
                    response = response_dict.get("response", "")
                    success = response_dict.get("success", True)
                    tool_content = json.dumps({
                        "response": response,
                        "success": success
                    })
                else:
                    raise ValueError(f"Unknown function: {function_name}")

            except TypeError as e:
                # Handle the case where arguments were incorrectly provided or missing
                logger.error(f"Error occurred during function call due to argument mismatch: {e}")
                tool_content = json.dumps({
                    "response": f"Argument mismatch error: {str(e)}",
                    "success": False
                })

            except Exception as e:
                logger.error(f"Error occurred while calling the function {function_name}: {e}")
                tool_content = json.dumps({
                    "response": "An error occurred during the function execution.",
                    "success": False
                })

            logger.info(f"Tool response: {tool_content}")
            # Prepare tool response message
            function_call_result_message = {
                "role": "tool",
                "content": tool_content,
                "tool_call_id": tool_call.id
            }

            # Update conversation history with function call and its response
            messages += [function_call_request_message, function_call_result_message]

            # Get response from the model based on function result
            start_time = time.time()
            function_response = client.chat.completions.create(
                model=model,
                messages=messages
            )
            end_time = time.time()
            latency = end_time - start_time
            logger.debug(f"Latency for answering after help from function call: {latency:.3f} seconds")

            bot_answer = function_response.choices[0].message.content
            print(f"Bot: {bot_answer}")
            messages.append({"role": "assistant", "content": bot_answer})

        elif "stop" in finish_reason:
            logger.warning("Answer NOT generated through function calling")
            print(f"Bot: {bot_answer}")
            messages.append({"role": "assistant", "content": bot_answer})


In [39]:

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model = 'gpt-4o'
tools = tools
with open('tools.json', 'w') as json_file:
    json.dump(tools, json_file, indent=4) 

system_prompt = general_system_prompt

conversation_loop(tools=tools, client=client, model=model, system_prompt=system_prompt)

Bot: Hello, this a Gold Cars assistant, how can I help you sir?
User: hello


2024-10-23 12:53:36.056 | DEBUG    | __main__:conversation_loop:33 - Latency for deciding between TextAnswer and FunctionCall: 0.550 seconds
2024-10-23 12:53:36.058 | WARNING  | __main__:conversation_loop:104 - Answer NOT generated through function calling


Bot: Hello! How can I assist you today?
User: i would like to buy a new car


2024-10-23 12:53:41.672 | DEBUG    | __main__:conversation_loop:33 - Latency for deciding between TextAnswer and FunctionCall: 0.808 seconds
2024-10-23 12:53:41.674 | WARNING  | __main__:conversation_loop:104 - Answer NOT generated through function calling


Bot: That's great! To assist you better, could you please tell me the brand of car you're interested in? Additionally, if you'd like to share your budget, it would help me provide better recommendations.
User: i am interested in a toyota, my full name is Alex Martinez


2024-10-23 12:53:55.130 | DEBUG    | __main__:conversation_loop:33 - Latency for deciding between TextAnswer and FunctionCall: 1.107 seconds
2024-10-23 12:53:55.131 | WARNING  | __main__:conversation_loop:104 - Answer NOT generated through function calling


Bot: Thank you, Alex Martinez. Could you please confirm the spelling of your name is A-L-E-X M-A-R-T-I-N-E-Z? Also, would you like to specify a maximum budget for your Toyota purchase?
User: yes, and i do not have a specific budget


2024-10-23 12:54:06.671 | DEBUG    | __main__:conversation_loop:33 - Latency for deciding between TextAnswer and FunctionCall: 1.489 seconds
2024-10-23 12:54:06.672 | INFO     | __main__:conversation_loop:44 - Tool call: ChatCompletionMessageToolCall(id='call_2YHjp7aX0uQ1Wey2Gx13UARb', function=Function(arguments='{"full_name":"Alex Martinez","brand":"Toyota","max_budget":100000,"current_time_of_year":"Summer"}', name='provide_car_purchase_info'), type='function')
2024-10-23 12:54:06.673 | INFO     | __main__:conversation_loop:45 - Function name: provide_car_purchase_info
2024-10-23 12:54:06.674 | INFO     | __main__:conversation_loop:46 - Function arguments: {'full_name': 'Alex Martinez', 'brand': 'Toyota', 'max_budget': 100000, 'current_time_of_year': 'Summer'}
2024-10-23 12:54:06.675 | INFO     | __main__:conversation_loop:78 - Tool response: {"response": "Hello Alex Martinez, here is the information about Toyota cars:\n- Brand: Toyota\n- Price Range: 20,000 - 35,000 USD\n- Descript

Bot: Hello Alex Martinez, here's some information about Toyota cars for you:

- **Price Range:** $20,000 - $35,000
- **Description:** Toyota offers a variety of reliable sedans, SUVs, and hybrids, known for fuel efficiency and long-lasting performance.
- **Current Time of Year:** Summer is ideal for finding deals, especially around Spring.

Based on your budget of up to $100,000, you might be interested in the Toyota Model X or Model Y. Let me know if there's anything else you need!
